In [ ]:
import manager
import cv2
import importlib
import numpy as np
import os

In [ ]:
importlib.reload(manager)
importlib.reload(manager.utils)

for file in os.listdir(os.fsencode("In")):
    filename = os.fsdecode(file)

    if not filename.endswith(".mp4"):
        continue

    cap1 = cv2.VideoCapture(f"In/{filename}")
    CamManagers = np.array([manager.CameraManager(cap1, f"Video {filename[:-4]}")])

    while len(CamManagers) > 0:
        for CamManager in CamManagers:
            res, out = CamManager.main()
            if res:
                continue
            
            CamManagers = CamManagers[CamManagers != CamManager]
            print(out)

In [ ]:
# Take photos for calibration
TIMED = True

import cv2
import os
from datetime import datetime
if TIMED: import time

cap = cv2.VideoCapture(1)

print("y - take image")
print("q - quit")

while True:
    ret, img = cap.read()
    if not ret: break
    cv2.imshow('img1',img)
    
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('y'):
        if TIMED:
            time.sleep(3)
            ret, img = cap.read()
            if not ret: break

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        filename = f"{timestamp}.jpg"
        filepath = os.path.join("calibration", filename)
        cv2.imwrite(filepath, img)

    elif key == ord('q'):
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break

cap.release()

In [ ]:
# Calibrate
import utils
import os
import cv2
import numpy as np

CamViewDepths = []

for i, file in enumerate(os.listdir(os.fsencode("calibration"))):
    filename = os.path.join("calibration", os.fsdecode(file))

    img = cv2.imread(filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    CamViewDepth = utils.GetCamViewDepth(img, verbose=False)
    if not CamViewDepth:
        print(i, "No person detected")
        continue
    
    CamViewDepths.append(CamViewDepth)

CamViewDepth = np.mean(CamViewDepths)
print(CamViewDepth)
print(CamViewDepths)

In [ ]:
# Main
CamViewDepth = 1844.149474379281
CALC_FPS = True
save = False
TestFile = "walk1.pkl" # trip walk faint sit
SaveGraphs = False

import cv2
import manager
import importlib # ==!== ==!== ==!== ==!== ==!== ==!==
importlib.reload(manager) # ==!== ==!== ==!== ==!== ==!== ==!==
if CALC_FPS: from datetime import datetime

cap = cv2.VideoCapture(1)
NetworkManager = manager.utils.NetworkManager("Matthew-Bird.local")
CameraManager = manager.CameraManager(cap, CamViewDepth, NetworkManager, f"Cap 1", SaveGraphs=SaveGraphs)

if CALC_FPS: t0 = datetime.now().timestamp()
try:
    while True:
        success, log = CameraManager.main(save=save, TestFile=TestFile)

        if CALC_FPS:
            t1 = datetime.now().timestamp()
            print(f"fps: {round(1/(t1-t0), 1)}")
            t0 = t1

        if not success:
            print(log)
            break
except KeyboardInterrupt:
    if CameraManager.SaveVideoOfIncident:
        CameraManager.UpdateVideoAsync(None, True)
    for body in CameraManager.bodies: body.save_graphs()
finally:
    if save: CameraManager.save(TestFile)
    cap.release()

In [ ]:
import cv2

cap = cv2.VideoCapture(1)

while True:
    ret, img = cap.read()
    cv2.imshow('img1',img)
    if cv2.waitKey(1) & 0xFF == ord('y'):
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break

cap.release()

In [ ]:
cv2.destroyAllWindows()
cv2.waitKey(1)

test velocity and y and y max calculations using graphs

find v thresholds for knee and head

test with actual falls